In [26]:
from dataloader import create_split_loaders
from model import GenericRNN
import torch

In [27]:
import torch.nn as nn
import copy
import os
from utility import *

config = {'chunk_size':100, 'type_number':93, 'hidden':100, 
          'learning_rate':0.001, 'early_stop':True, 'increase_limit':3, 
          'epoch_num':1, 'N':50, 'M':100, 'seed':1, 'model':'LSTM', 'model_path':'model_weights'}

def train(config):
    use_cuda = torch.cuda.is_available()
    if use_cuda:
        computing_device = torch.device("cuda")
        extras = {"num_workers": 1, "pin_memory": True}
        print("CUDA is supported")
    else: # Otherwise, train on the CPU
        computing_device = torch.device("cpu")
        extras = False
        print("CUDA NOT supported")
    
    seed = config['seed']
    torch.manual_seed(seed)
    
    # the size of every chunk
    chunk_size = config['chunk_size']
    # number of types, it is a constant
    type_number = config['type_number']
    # number of features in hidden layer
    hidden = config['hidden']
    # learning rate
    learning_rate = config['learning_rate']
    # whether we use early stop
    early_stop = config['early_stop']
    # after validation loss increase how many times do we stop training
    increase_limit = config['increase_limit']
    # number of epoch
    epoch_num = config['epoch_num']
    # receive train, validation, test data
    train, valid, test, c_to, one_to = create_split_loaders(chunk_size,extras)
    
    # construct network
    net = GenericRNN(type_number, hidden, type_number, config['model'])
    # if model already exists, then load the previous one
    if os.path.exists(config['model_path']+'.pkl'):
        print('loading previous model...')
        load_net_state(net, config['model_path']+'.pkl')
    net = net.to(computing_device)
    
    # use cross entropy loss
    criterion = nn.BCELoss()
    # keep tracking of the traininig loss
    training_record = []
    # keep tracking of the validation loss
    validation_record = []
    
    # Using Adam
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
    # if model already exists, then load the previous optimizer state
    prev_total_loss, prev_avg_minibatch_loss, prev_val_loss = [], [], []
    if os.path.exists(config['model_path']+'.pkl'):
        print('loading optimizer state...')
        load_opt_state(optimizer, config['model_path']+'.pkl')
        # notice when saving prev_val_loss, we ignored the first val_loss
        prev_total_loss, prev_avg_minibatch_loss, prev_val_loss = load_loss(config['model_path']+'.pkl')
    
    total_loss = [] + prev_total_loss
    avg_minibatch_loss = [] + prev_avg_minibatch_loss
    val_loss = [1000000000] + prev_val_loss #assume large error at the begining

    last_valid = float('inf')
    best_net = -1
    # after how many batches do we record the training loss
    N = config['N']
    # after how many batches do we examine whether validation loss increase
    M = config['M']
    # store best loss
    best_loss = float('inf')
    increasement = 0
    for epoch in range(epoch_num):
        count = 0
        average_loss = 0
        state_0 = None
        for minibatch in train:
            count += 1
            if minibatch[0].size()[0] != chunk_size:
                break
            optimizer.zero_grad()
            train_batch = minibatch[0]
            target_batch = minibatch[1]
            train_batch = train_batch.to(computing_device)
            target_batch = target_batch.to(computing_device)
            predict_batch, state_0 = net(train_batch, state_0)
            if isinstance(state_0, tuple):
                state_0 = list(state_0)
                for i in range(len(state_0)):
                    state_0[i] = state_0[i].detach()
                state_0 = tuple(state_0)
            else:
                state_0 = state_0.detach()
            loss = criterion(predict_batch, target_batch)
            loss.backward()
            optimizer.step()
            total_loss.append(loss.item())
            average_loss += loss.item()
            if count % N == 0:
                training_record.append(average_loss / N)
                avg_minibatch_loss.append(average_loss / N)
                average_loss = 0
                print('keep tracking of training error:')
                print(training_record)
            # validation 
            if count % M == 0:
                with torch.no_grad():
                    loss_val = 0
                    count_val = 0
                    state_0 = None
                    for val in valid:
                        count_val += 1
                        if val[0].size()[0] != chunk_size:
                            break
                        valid_batch = val[0].to(computing_device)
                        valid_target = val[1].to(computing_device)
                        valid_predict, state_0 = net(valid_batch, state_0)
                        loss_val += criterion(valid_predict, valid_target)
                    loss_val /= count_val
                    val_loss.append(loss_val.item())
                    validation_record.append(loss_val.item())
                    print('keep tracking of validation error')
                    print(validation_record)
                    if loss_val < best_loss:
                        print('best model is updated')
                        best_loss = loss_val
                        best_net = copy.deepcopy(net)
                save_state(best_net, optimizer, total_loss, avg_minibatch_loss, val_loss[1:], \
                           seed, config['model_path']+'.pkl')
                if early_stop:
                    if loss_val > last_valid:
                        increasement += 1
                    else:
                        increasement = 0
                    last_valid = loss_val
                    if increasement >= increase_limit:
                        break
        if early_stop:
            if increasement >= increase_limit:
                break

In [28]:
train(config)

CUDA NOT supported
loading previous model...
loading optimizer state...
keep tracking of training error:
[0.04477706223726272]
keep tracking of training error:
[0.04477706223726272, 0.03850761767476797]
keep tracking of validation error
[0.04889550805091858]
best model is updated
keep tracking of training error:
[0.04477706223726272, 0.03850761767476797, 0.04147551707923412]
keep tracking of training error:
[0.04477706223726272, 0.03850761767476797, 0.04147551707923412, 0.03873007643967867]
keep tracking of validation error
[0.04889550805091858, 0.047383248805999756]
best model is updated
keep tracking of training error:
[0.04477706223726272, 0.03850761767476797, 0.04147551707923412, 0.03873007643967867, 0.03543800260871649]
keep tracking of training error:
[0.04477706223726272, 0.03850761767476797, 0.04147551707923412, 0.03873007643967867, 0.03543800260871649, 0.0375898901745677]
keep tracking of validation error
[0.04889550805091858, 0.047383248805999756, 0.04709471017122269]
best mo